In [21]:
import importlib
import scraper

importlib.reload(scraper)  # Forces reloading of the scraper module

from scraper import fetch_sold_listings, get_listing_details, debug_pagination

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()

# Automatically match ChromeDriver with installed Chrome version
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://google.com")  # Test if Chrome opens correctly

print("Chrome opened successfully!")
driver.quit()

Chrome opened successfully!


In [ ]:
# Run the scraper
url = "https://housesigma.com/on/sold/map/?status=sold&lat=43.715564&lon=-79.418602&zoom=10&page=1&view=list"

# debug_pagination(url)

fetch_sold_listings(url)

Opening HouseSigma...
✅ Saved full page HTML: full_page_source.html


In [14]:
prices_df = get_listing_details("logged_in_page_source.html")

print(prices_df)

Reading html file
   Asking Price  Sold Price                                            Address
0      $998,000    $975,000   623 - 88 Colgate Ave , Toronto - South Riverdale
1      $639,900    $635,000         524 E - 2908 Highway 7 , Vaughan - Concord
2    $1,195,100  $1,150,000  1409 - 28 Freeland St , Toronto - Waterfront C...
3      $989,000    $950,000  507B - 660 Sheppard Ave E , North York - Bayvi...
4    $1,100,000  $1,000,000       264 Nairn Ave , Toronto - Caledonia-Fairbank
5    $1,089,000  $1,030,000  176 Rosemount Ave , Toronto - Corso Italia-Dav...
6    $1,595,000  $1,585,000         198 LAUDER Ave , Toronto - Oakwood Village
7      $749,999    $733,000  19 Dee Ave , North York - Humberlea-Pelmo Park W4
8      $499,900    $470,100      509 - 10 Meadowglen Pl , Scarborough - Woburn
9      $579,900    $550,000  4909 - 181 Dundas St E , Toronto - Church-Yong...
10   $1,395,000  $1,400,000  39B Newington Cres , Etobicoke - Eringate-Cent...
11     $545,999    $537,500  1415 